In [8]:
from wikidata2df import wikidata2df

query = """

SELECT ?geneLabel ?cellTypeLabel ?processLabel
WHERE 
{
  ?protein wdt:P682 wd:Q1456827. # protein molecular process neurogenesis
  ?protein wdt:P702 ?gene.       # protein encoded by gene
  
  {?gene wdt:P31 wd:Q277338.}    # gene is an instance of a pseudogene 
  UNION                          # or
  {?gene wdt:P31 wd:Q7187.}      # gene is an instance of a gene
  ?gene wdt:P703 wd:Q15978631.   # gene is found in taxon Homo sapiens
  
  ?cellType wdt:P8872 ?gene.     # cell type has marker gene
  
  ?cellType rdfs:label ?cellTypeLabel.
  ?gene   rdfs:label ?geneLabel.
  wd:Q1456827 rdfs:label ?processLabel

  FILTER(LANG(?cellTypeLabel) = "en")
  FILTER(LANG(?geneLabel) = "en")
  FILTER(LANG(?processLabel) = "en")

}
LIMIT 10
"""

neurogenesis = wikidata2df(query)

In [9]:
from tabulate import tabulate
print(tabulate(neurogenesis.head(10), tablefmt="pipe", headers="keys",showindex="never"))


| geneLabel   | processLabel   | cellTypeLabel                   |
|:------------|:---------------|:--------------------------------|
| EPHB1       | neurogenesis   | human oligodendrocyte           |
| EPHB1       | neurogenesis   | human osteoclast                |
| OMP         | neurogenesis   | human purkinje neuron           |
| OMP         | neurogenesis   | human olfactory epithelial cell |
| OMP         | neurogenesis   | human neuron                    |
| PCSK9       | neurogenesis   | human delta cell                |
| PCSK9       | neurogenesis   | human loop of Henle cell        |
| CXCR4       | neurogenesis   | human b cell                    |
| CXCR4       | neurogenesis   | human T cell                    |
| CXCR4       | neurogenesis   | human nk cell                   |


In [10]:
query = """

SELECT ?cellTypeLabel ?geneLabel ?diseaseLabel 
WHERE 
{
  wd:Q11085 wdt:P2293 ?diseaseGene.  # Parkinson's disease --> genetic association --> gene
  ?cellType wdt:P8872 ?diseaseGene. # Cell type --> has marker --> gene
  
  ?cellType rdfs:label ?cellTypeLabel.
  wd:Q11085 rdfs:label ?diseaseLabel.
  ?diseaseGene   rdfs:label ?geneLabel.

  FILTER(LANG(?cellTypeLabel) = "en")
  FILTER(LANG(?diseaseLabel) = "en")
  FILTER(LANG(?geneLabel) = "en")
}
LIMIT 5
"""

# https://w.wiki/yQD

parkinson = wikidata2df(query)

In [12]:
print(tabulate(parkinson.head(10), tablefmt="pipe", headers="keys",showindex="never"))

| geneLabel   | diseaseLabel        | cellTypeLabel    |
|:------------|:--------------------|:-----------------|
| BST1        | Parkinson's disease | human b cell     |
| BST1        | Parkinson's disease | human neutrophil |
| RIT2        | Parkinson's disease | human neuron     |
| SH3GL2      | Parkinson's disease | human alpha cell |
| SH3GL2      | Parkinson's disease | human beta cell  |


In [13]:
query = """

SELECT ?cellTypeLabel ?diseaseLabel 
(COUNT(DISTINCT ?diseaseGene) AS ?count) 
(GROUP_CONCAT(DISTINCT ?geneLabel; SEPARATOR=", ") AS ?genes)
WHERE 
{
  wd:Q101405087 wdt:P8872 ?diseaseGene .    # human pancreatic beta cell -->  has marker -->  gene
  ?disease wdt:P2293 ?diseaseGene .         # disease --> genetic association --> gene
  
  wd:Q101405087 rdfs:label ?cellTypeLabel .
  ?disease rdfs:label ?diseaseLabel .
  ?diseaseGene   rdfs:label ?geneLabel .

  FILTER(LANG(?cellTypeLabel) = "en")
  FILTER(LANG(?diseaseLabel) = "en")
  FILTER(LANG(?geneLabel) = "en")
  
 }

GROUP BY ?diseaseLabel ?cellTypeLabel ORDER BY DESC(?count)
LIMIT 5
"""

# https://w.wiki/yQE

beta_cell = wikidata2df(query)

In [14]:
print(tabulate(beta_cell.head(10), tablefmt="pipe", headers="keys",showindex="never"))

| diseaseLabel        | genes                   |   count | cellTypeLabel   |
|:--------------------|:------------------------|--------:|:----------------|
| obesity             | PCSK2, ADCYAP1, SLC30A8 |       3 | human beta cell |
| type 2 diabetes     | SLC30A8, TGFBR3         |       2 | human beta cell |
| Parkinson's disease | SH3GL2                  |       1 | human beta cell |
| asthma              | SLC30A8                 |       1 | human beta cell |
| aniridia            | PAX6                    |       1 | human beta cell |
